# 4. SORU

## Mounting the Google Drive Connection 

In [ ]:
# COLAB GOOGLE DRIVE CONNECTION/ MOUNT
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

  # MOUNT
if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/stackoverflow"
  
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root
print('='*40)
%pwd

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/stackoverflow  exists.

Colab: Changing directory to  /content/gdrive/My Drive/stackoverflow
/content/gdrive/My Drive/stackoverflow


'/content/gdrive/My Drive/stackoverflow'

## Import necessary moduls


In [ ]:
# Import necessary moduls
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Import numpy and pandas
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import *

### install dependencies 

In [ ]:
!pip install textsearch
!pip install contractions
!pip install textsearch
import nltk
nltk.download('punkt')
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load and View Dataset 

In [ ]:
# Readind the csv file
df= pd.read_csv("stack.csv")
df[0:3]

,Unnamed: 0,tags,owner,title,body,score,last_activity_date,creation_date,question_id,view_count
0,1328,"['python', 'pip', 'tox']","{'reputation': 38, 'user_id': 17981284, 'displ...",tox refuses to use deps setting in py38 and va...,I have a Django project that i'm trying to set...,0,2022-04-05 03:47:16,2022-04-01 00:22:22,71700514,467
1,2385,"['python', 'pip', 'easyocr']","{'reputation': 11, 'user_id': 18649779, 'displ...",easyocr installation error when install pillow,"I'm trying to install the easyocr library, but...",-1,2022-04-03 14:42:33,2022-04-01 00:27:03,71700531,132
2,2175,"['python', 'reduce', 'numba', 'jit', 'jax']","{'reputation': 79, 'user_id': 18649992, 'displ...",JAX(XLA) vs Numba(LLVM) Reduction,Is it possible to make CPU only reductions wit...,2,2022-04-03 23:01:43,2022-04-01 02:15:55,71701041,713


In [ ]:
df['owner'].values[0:3]

array(["{'reputation': 38, 'user_id': 17981284, 'display_name': 'dafrandle'}",
       "{'reputation': 11, 'user_id': 18649779, 'display_name': 'JHOSEF NASCIMENTO'}",
       "{'reputation': 79, 'user_id': 18649992, 'display_name': 'DavidJ'}"],
      dtype=object)

## Pre-processing

In [ ]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['title'].values))
len(norm_corpus)

32890

In [ ]:
norm_corpus[0:5]

array(['tox refuses use deps setting py38 various pip problems',
       'easyocr installation error install pillow',
       'jaxxla vs numballvm reduction',
       'problem matplotlibpyplot vscodes',
       'typeerror init missing 1 required positional argument 39units39 coding neural net'],
      dtype='<U148')


## Extracing Features (TF-IDF)
> ***FEATURES: TF-IDF*** : Term Frequency-Inverse Document Frequency




In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

stop_words = stop_words + ['one', 'two', 'three', 'four', 'five', 'get']
cv = CountVectorizer(ngram_range=(1, 2), min_df=0.001, max_df=0.99, stop_words=stop_words)
features = cv.fit_transform(norm_corpus)
features.shape

(32890, 1163)

##Clustering


> ***K-Means ***



K-Means is one of the most popular "clustering" algorithms. K-means stores $k$ centroids that it uses to define clusters. A point is considered to be in a particular cluster if it is closer to that cluster's centroid than any other centroid.

K-Means finds the best centroids by alternating between 

(1) assigning data points to clusters based on the current centroids 
(2) chosing centroids (points which are the center of a cluster) based on the current assignment of data points to clusters.

till convergence..

![](https://i.imgur.com/42n9uvR.png)

Features we are using here are BOW based features

In [ ]:
from sklearn.cluster import KMeans

NUM_CLUSTERS = 2
km = KMeans(n_clusters=NUM_CLUSTERS, max_iter=10000, n_init=100, random_state=42).fit(features)
km

KMeans(max_iter=10000, n_clusters=2, n_init=100, random_state=42)

In [ ]:
yh =km.predict(features)
yh[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0],
      dtype=int32)

In [ ]:
df['Cluster Label'] = km.labels_

In [ ]:
df[0:3]

,Unnamed: 0,tags,owner,title,body,score,last_activity_date,creation_date,question_id,view_count,Cluster Label
0,1328,"['python', 'pip', 'tox']","{'reputation': 38, 'user_id': 17981284, 'displ...",tox refuses to use deps setting in py38 and va...,I have a Django project that i'm trying to set...,0,2022-04-05 03:47:16,2022-04-01 00:22:22,71700514,467,0
1,2385,"['python', 'pip', 'easyocr']","{'reputation': 11, 'user_id': 18649779, 'displ...",easyocr installation error when install pillow,"I'm trying to install the easyocr library, but...",-1,2022-04-03 14:42:33,2022-04-01 00:27:03,71700531,132,0
2,2175,"['python', 'reduce', 'numba', 'jit', 'jax']","{'reputation': 79, 'user_id': 18649992, 'displ...",JAX(XLA) vs Numba(LLVM) Reduction,Is it possible to make CPU only reductions wit...,2,2022-04-03 23:01:43,2022-04-01 02:15:55,71701041,713,0


# Each entry were classified into 2 cluster labels: 

> As ***0 (dissimilar)*** and ***1 (similar)*** based on TF-IDF feature similarity.



##   Owners can then be classified from those classes common characteristics/properties such favarite tag topics, view counts etc.



## Cluster 1

In [ ]:
df_1 = df.loc[ df['Cluster Label'] == 1 ]
print ( df_1['title'][0:10] )

8     Cpanel - I am trying to install pillow module ...
11    Job type(Full Time , Part Time) detection with...
12    Job type(Full Time , Part Time) detection with...
13             ignoring data using ttp module in python
17    Python and spreadsheet: given the user input, ...
22      python dictionary array of callbacks with types
24    Uploading images to a windows server using python
25    Tensorflow initializing in Python without bein...
26    Is there a python library to use Choco solver ...
27    Rewriting MATLAB code with nested loops in Pyt...
Name: title, dtype: object


In [ ]:
print(df_1.nunique())

Unnamed: 0            7907
tags                  5607
owner                 7247
title                 8269
body                  8266
score                   15
last_activity_date    8263
creation_date         8256
question_id           8267
view_count             338
Cluster Label            1
dtype: int64


## Cluster 0

In [ ]:
df_0 = df.loc[ df['Cluster Label'] == 0 ]
print ( df_1['title'][0:10] )

8     Cpanel - I am trying to install pillow module ...
11    Job type(Full Time , Part Time) detection with...
12    Job type(Full Time , Part Time) detection with...
13             ignoring data using ttp module in python
17    Python and spreadsheet: given the user input, ...
22      python dictionary array of callbacks with types
24    Uploading images to a windows server using python
25    Tensorflow initializing in Python without bein...
26    Is there a python library to use Choco solver ...
27    Rewriting MATLAB code with nested loops in Pyt...
Name: title, dtype: object


In [ ]:
df_0.nunique()[2]

18516

In [ ]:
print(df_0.nunique())

Unnamed: 0            19537
tags                  14051
owner                 18516
title                 23549
body                  23529
score                    24
last_activity_date    23434
creation_date         23451
question_id           23535
view_count              572
Cluster Label             1
dtype: int64


## Distribution of the Owners into Two-Classes:



In [ ]:
print('Number of unique entry owner for Cluster_0:',df_0.nunique()[2] )
print('\nNumber of unique entry owner for Cluster_1:',df_1.nunique()[2] )

Number of unique entry owner for Cluster_0: 18516

Number of unique entry owner for Cluster_1: 7247


## Tag distribution of the df_1



In [ ]:
# Assigning all tag column values
sentences_distinct_tags_1 = df_1['tags'].values

In [ ]:
tokenizer = Tokenizer(num_words = 33000, oov_token="<OOV>", filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n', split="," )
tokenizer.fit_on_texts(sentences_distinct_tags_1)
word_index_1 = tokenizer.word_index

list_word_index_1=list(word_index_1)
print(list_word_index_1[0:15])

['<OOV>', "'python'", " 'python-3", " 'pandas'", "x'", " 'dataframe'", " 'list'", " 'selenium'", " 'numpy'", " 'json'", " 'dictionary'", " 'tkinter'", " 'matplotlib'", " 'web-scraping'", " 'csv'"]


## Tag distribution of the df_0

In [ ]:
# Assigning all tag column values
sentences_distinct_tags_0 = df_0['tags'].values

In [ ]:
tokenizer = Tokenizer(num_words = 33000, oov_token="<OOV>", filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n', split="," )
tokenizer.fit_on_texts(sentences_distinct_tags_0)
word_index_0 = tokenizer.word_index

list_word_index_0=list(word_index_0)
print(list_word_index_0[0:15])

['<OOV>', "'python'", " 'pandas'", " 'dataframe'", " 'python-3", "x'", " 'django'", " 'machine-learning'", " 'tensorflow'", " 'deep-learning'", " 'numpy'", " 'keras'", " 'matplotlib'", " 'pytorch'", " 'list'"]


# PERFORMANCE EVALUATION OF THE CLUSTERING:

## Evaluation of the Clusters from the ***User Behavior Viewpoint***:


> The Tags ***"machine-learning"*** and ***"deep-learning"*** are only frequent (first 15 item) for the owners (authors) whom classified to the **Cluster_0**. 

> ***So, we may infere that those owners in the***  ***Cluster_0 work in the filed of the AI/ML.***






In [ ]:
print('15 Most frequent tags of the owner whom assigned to Cluster_1',list_word_index_1[0:15])
print('='*40)
print('15 Most frequent tags of the owner whom assigned to Cluster_0',list_word_index_0[0:15])
print('='*40)

15 Most frequent tags of the owner whom assigned to Cluster_1 ['<OOV>', "'python'", " 'python-3", " 'pandas'", "x'", " 'dataframe'", " 'list'", " 'selenium'", " 'numpy'", " 'json'", " 'dictionary'", " 'tkinter'", " 'matplotlib'", " 'web-scraping'", " 'csv'"]
15 Most frequent tags of the owner whom assigned to Cluster_0 ['<OOV>', "'python'", " 'pandas'", " 'dataframe'", " 'python-3", "x'", " 'django'", " 'machine-learning'", " 'tensorflow'", " 'deep-learning'", " 'numpy'", " 'keras'", " 'matplotlib'", " 'pytorch'", " 'list'"]


### We may infer that ML/DL Topics is the main reason for the increase in the view counts since nearly all top 15 tags of the two clusters are nearly the same except ML, DL tags.

In [ ]:
print('Mean of the view_counts in Cluster_1',np.mean(df_1['view_count']))
print('='*40)
print('Mean of the view_counts in Cluster_0',np.mean(df_0['view_count']))
print('='*40)



Mean of the view_counts in Cluster_1 58.82021531100479
Mean of the view_counts in Cluster_0 64.89339584182633
